In [ ]:
# !apt-get update 
# !pip3 install pybullet
# !apt-get install -y ffmpeg

In [ ]:
import pybullet as p
import time
import numpy as np
import os
import _utils as u
from _utils import geom_shape, xyz, s
import sys

SPH = p.GEOM_SPHERE
BOX = p.GEOM_BOX
CYL = p.GEOM_CYLINDER
CAP = p.GEOM_CAPSULE

VARIABLE = 0.04
# joint coefficient
j = 0.25

u.init_window()
# b_p = u.coordinates(0,0.9,0)
# base= u.geom_shape(BOX, height = VARIABLE, coeff_P = [1,1,1])
# body_Mass = 0.1
b_p = u.coordinates(0,0,0)
base= u.geom_shape(BOX, height = VARIABLE, coeff_P = [10,10,0.1])
body_Mass = 20000

#-------------------------------------------------------------------------------------------------------------
M_Pelvis = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
M_TorsoS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_TorsoS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_TorsoF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_TorsoF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_TorsoR = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_TorsoR = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
#-------------------------------------------------------------------------------------------------------------
M_Neck   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_Neck   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_HeadR  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_HeadR  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_HeadF  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_HeadF  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
#-------------------------------------------------------------------------------------------------------------
M_ShldrF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_ShldrF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_ShldrS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_ShldrS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_ElbowR = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_ElbowR = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_Elbow  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_Elbow  = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
#-------------------------------------------------------------------------------------------------------------
M_HipF   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_HipF   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_HipS   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_HipS   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_HipR   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_HipR   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_Knee   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_Knee   = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_AnkleF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = 1)
J_AnkleF = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_AnkleS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
J_AnkleS = u.geom_shape(SPH, radius = VARIABLE, coeff_R = j)
M_Foot   = u.geom_shape(BOX, height = VARIABLE, coeff_P = [1,3,0.25])

In [ ]:
M_Pelvis

In [ ]:
limiter = 60

linkCollisionShapeIndices=[
    M_Pelvis, 
    J_TorsoS, M_TorsoS, J_TorsoF, M_TorsoF, J_TorsoR, M_TorsoR, # 7 
    J_Neck,   M_Neck,                                           # 9
    J_HeadR,  M_HeadR,  J_HeadF,  M_HeadF,                      # 13
    J_ShldrF, M_ShldrF, J_ShldrS, M_ShldrS,                     # 17
    J_ElbowR, M_ElbowR, J_Elbow,  M_Elbow,                      # 21
    J_ShldrF, M_ShldrF, J_ShldrS, M_ShldrS,                     # 25
    J_ElbowR, M_ElbowR, J_Elbow,  M_Elbow,                      # 29

    J_HipF,   M_HipF,   J_HipS,   M_HipS,   J_HipR,   M_HipR,   # 35
    J_Knee,   M_Knee,                                           # 37
    J_AnkleF, M_AnkleF, J_AnkleS, M_AnkleS,                     # 41
              M_Foot,                                           # 42
    J_HipF,   M_HipF,   J_HipS,   M_HipS,   J_HipR,   M_HipR,   # 48
    J_Knee,   M_Knee,                                           # 50
    J_AnkleF, M_AnkleF, J_AnkleS, M_AnkleS,                     # 54
              M_Foot,                                           # 55
][:limiter]
#-------------------------------------------------------------------------------------------------------------
m = 0.00000001
link_Masses=[
    # backbone
       u.MASSES['Pelvis'     ].mass, 
    m, u.MASSES['TorsoS'     ].mass, m, u.MASSES['TorsoF'     ].mass, m, u.MASSES['TorsoR'].mass,
    # neck and head
    m, u.MASSES['Neck'       ].mass, 
    m, u.MASSES['HeadR'      ].mass, m, u.MASSES['HeadF'      ].mass,
    # Right Arm
    m, u.MASSES['R_ShoulderF'].mass, m, u.MASSES['R_ShoulderS'].mass, 
    m, u.MASSES['R_ElbowR'   ].mass, m, u.MASSES['R_Elbow'    ].mass,
    # Left Arm
    m, u.MASSES['L_ShoulderF'].mass, m, u.MASSES['L_ShoulderS'].mass, 
    m, u.MASSES['L_ElbowR'   ].mass, m, u.MASSES['L_Elbow'    ].mass,
    # Right Leg
    m, u.MASSES['R_HipF'     ].mass, m, u.MASSES['R_HipS'     ].mass, m, u.MASSES['R_HipR'].mass, 
    m, u.MASSES['R_Knee'     ].mass,
    m, u.MASSES['R_AnkleF'   ].mass, m, u.MASSES['R_AnkleS'   ].mass,
       u.MASSES['R_Foot'     ].mass,
    # Left Leg
    m, u.MASSES['L_HipF'     ].mass, m, u.MASSES['L_HipS'     ].mass, m, u.MASSES['L_HipR'].mass, 
    m, u.MASSES['L_Knee'     ].mass,
    m, u.MASSES['L_AnkleF'   ].mass, m, u.MASSES['L_AnkleS'   ].mass,
       u.MASSES['L_Foot'     ].mass,
][:limiter]
#-------------------------------------------------------------------------------------------------------------
#link positions wrt the link they are attached to
m2j = ['m','j']; j2m = ['j','m']; m2m = ['m','m']; d2m = ['d','m']; m2d = ['m','d']
linkPositions = [
    xyz(d2m,'CMS_Coords', 'Pelvis',    data = b_p),                              # Pelvis
    # xyz(m2d,'Pelvis',     'CMS_Coords', data = b_p),                              # Pelvis
    xyz(m2j,'Pelvis',     'TorsoS'     ), xyz(j2m, 'TorsoS',     'TorsoS'     ), # Pelvis      + TorsoS
    xyz(m2j,'TorsoS',     'TorsoF'     ), xyz(j2m, 'TorsoF',     'TorsoF'     ), # TorsoS      + TorsoF 
    xyz(m2j,'TorsoF',     'TorsoR'     ), xyz(j2m, 'TorsoR',     'TorsoR'     ), # TorsoF      + TorsoR 
    xyz(m2j,'TorsoR',     'Neck'       ), xyz(j2m, 'Neck'  ,     'Neck'       ), # TorsoR      + Neck 
    xyz(m2j,'Neck',       'HeadR'      ), xyz(j2m, 'HeadR' ,     'HeadR'      ), # Neck        + HeadR
    xyz(m2j,'HeadR',      'HeadF'      ), xyz(j2m, 'HeadF' ,     'HeadF'      ), # HeadR       + HeadF
    xyz(m2j,'TorsoR',     'R_ShoulderF'), xyz(j2m, 'R_ShoulderF','R_ShoulderF'), # TorsoR      + R_ShoulderF
    xyz(m2j,'R_ShoulderF','R_ShoulderS'), xyz(j2m, 'R_ShoulderS','R_ShoulderS'), # R_ShoulderF + R_ShoulderF
    xyz(m2j,'R_ShoulderF','R_ElbowR'   ), xyz(j2m, 'R_ElbowR',   'R_ElbowR'   ), # R_ShoulderF + R_ElbowR
    xyz(m2j,'R_ElbowR',   'R_Elbow'    ), xyz(j2m, 'R_Elbow',    'R_Elbow'    ), # R_ElbowR    + R_Elbow
    xyz(m2j,'TorsoR',     'L_ShoulderF'), xyz(j2m, 'L_ShoulderF','L_ShoulderF'), # TorsoR      + L_ShoulderF
    xyz(m2j,'L_ShoulderF','L_ShoulderS'), xyz(j2m, 'L_ShoulderS','L_ShoulderS'), # L_ShoulderF + L_ShoulderF
    xyz(m2j,'L_ShoulderF','L_ElbowR'   ), xyz(j2m, 'L_ElbowR',   'L_ElbowR'   ), # L_ShoulderF + L_ElbowR
    xyz(m2j,'L_ElbowR',   'L_Elbow'    ), xyz(j2m, 'L_Elbow',    'L_Elbow'    ), # L_ElbowR    + L_Elbow
    xyz(m2j,'Pelvis',     'R_HipF'     ), xyz(j2m, 'R_HipF',     'R_HipF'     ), # Pelvis      + R_HipF
    xyz(m2j,'R_HipF',     'R_HipS'     ), xyz(j2m, 'R_HipS',     'R_HipS'     ), # R_HipF      + R_HipS
    xyz(m2j,'R_HipS',     'R_HipR'     ), xyz(j2m, 'R_HipR',     'R_HipR'     ), # R_HipS      + R_HipR
    xyz(m2j,'R_HipR',     'R_Knee'     ), xyz(j2m, 'R_Knee',     'R_Knee'     ), # R_HipR      + R_Knee
    xyz(m2j,'R_Knee',     'R_AnkleF'   ), xyz(j2m, 'R_AnkleF',   'R_AnkleF'   ), # R_Knee      + R_AnkleF
    xyz(m2j,'R_AnkleF',   'R_AnkleS'   ), xyz(j2m, 'R_AnkleS',   'R_AnkleS'   ), # R_AnkleF    + R_AnkleS
    xyz(m2m,'R_AnkleS',   'R_Foot'     ),                                        # R_Foot
    xyz(m2j,'Pelvis',     'L_HipF'     ), xyz(j2m, 'L_HipF',     'L_HipF'     ), # Pelvis      + L_HipF
    xyz(m2j,'L_HipF',     'L_HipS'     ), xyz(j2m, 'L_HipS',     'L_HipS'     ), # L_HipF      + L_HipS
    xyz(m2j,'L_HipS',     'L_HipR'     ), xyz(j2m, 'L_HipR',     'L_HipR'     ), # L_HipS      + L_HipR
    xyz(m2j,'L_HipR',     'L_Knee'     ), xyz(j2m, 'L_Knee',     'L_Knee'     ), # L_HipR      + L_Knee
    xyz(m2j,'L_Knee',     'L_AnkleF'   ), xyz(j2m, 'L_AnkleF',   'L_AnkleF'   ), # L_Knee      + L_AnkleF
    xyz(m2j,'L_AnkleF',   'L_AnkleS'   ), xyz(j2m, 'L_AnkleS',   'L_AnkleS'   ), # L_AnkleF    + L_AnkleS
    xyz(m2m,'L_AnkleS',   'L_Foot'     ),                                        # L_Foot
][:limiter]
#-------------------------------------------------------------------------------------------------------------
#indices determine for each link which other link it is attached to
# for example 3rd index = 2 means that the front left knee jjoint is attached to the front left hip
indices=[
    # pelvis,
           0,
    # j_tr_s, m_tr_s, j_tr_f, m_tr_f, j_tr_r, m_tr_r,
           1,      2,      3,      4,      5,      6,
    # j_neck, m_neck, j_hd_r, m_hd_r, j_hd_f, m_hd_f,
           7,      8,      9,     10,     11,     12,
    # j_ShlF, m_ShlF, j_ShlS, m_ShlS, j_ElbR, m_ElbR, j_Elbw, m_Elbw,
           7,     14,     15,     16,     17,     18,     19,     20,
           7,     22,     23,     24,     25,     26,     27,     28,
    # j_HipF, m_HipF, j_HipS, m_HipS, j_HipR, m_HipR, j_Knee, m_Knee, j_ankF, m_ankF, j_ankS, m_ankS, m_Foot,
           1,     30,     31,     32,     33,     34,     35,     36,     37,     38,     39,     40,     41,
           1,     43,     44,     45,     46,     47,     48,     49,     50,     51,     52,     53,     54,
][:limiter]
#-------------------------------------------------------------------------------------------------------------
# Most joint are revolving. The prismatic joints are kept fixed for now
prsm = p.JOINT_PRISMATIC
rvlt = p.JOINT_REVOLUTE
jointTypes=[
    # pelvis
        prsm,
    # j_tr_s, m_tr_s, j_tr_f, m_tr_f, j_tr_r, m_tr_r,
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,
    # j_neck, m_neck, j_hd_r, m_hd_r, j_hd_f, m_hd_f,
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,
    # j_ShlF, m_ShlF, j_ShlS, m_ShlS, j_ElbR, m_ElbR, j_Elbw, m_Elbw,
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,# Right Arm
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,#  Left Arm
    # j_HipF, m_HipF, j_HipS, m_HipS, j_HipR, m_HipR, j_Knee, m_Knee, j_ankF, m_ankF, j_ankS, m_ankS, m_Foot,
    # Right Leg
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   prsm, 
    #  Left Leg
        rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   rvlt,   prsm,   prsm, 
     
][:limiter]
#-------------------------------------------------------------------------------------------------------------
#revolution axis for each revolving joint
static = [0,0,0]
axis=[
    static, 
    #j_tr_s, m_tr_s, j_tr_f, m_tr_f, j_tr_r, m_tr_r,
    [0,1,0], static,[-1,0,0], static,[0,0,-1], static,
    #j_neck, m_neck, j_hd_r, m_hd_r, j_hd_f, m_hd_f,
    [-1,0,0], static,[0,0,-1], static,[-1,0,0], static,
    #j_ShlF, m_ShlF, j_ShlS, m_ShlS, j_ElbR, m_ElbR, j_Elbw, m_Elbw,
    [-1,0,0], static,[0,1,0], static,[0,0,1], static,[1,0,0], static, # Right Arm
    [-1,0,0], static,[0,1,0], static,[0,0,1], static,[1,0,0], static, #  Left Arm
    #j_HipF, m_HipF, j_HipS, m_HipS, j_HipR, m_HipR, 
    #j_Knee, m_Knee, j_ankF, m_ankF, j_ankS, m_ankS, m_Foot,
    [-1,0,0], static,[ 0,1,0], static,[0,0,-1], static,
    [-1,0,0], static,[-1,0,0], static,[0,1, 0], static, static,#Right Leg
    [-1,0,0], static,[ 0,1,0], static,[0,0,-1], static,
    [-1,0,0], static,[-1,0,0], static,[0,1, 0], static, static,# Left Leg
][:limiter]

# ARMS
Torso  = [1,3,5]; NandH  = [7,9,11]; 
ShldrF = [13,21]; ShldrS = [15,23]
ElbowR = [17,25]; Elbow  = [19,27];  
HipF   = [29,42]; HipS   = [31,44]
HipR   = [33,46]; 
Knee   = [35,48];  
AnkleF = [37,50]; AnkleS = [39,52]



# BLOCK Init
top = ShldrF + ShldrS + ElbowR + Elbow + Torso  + NandH
bot = HipF   + HipS   + HipR   + Knee  + AnkleF + AnkleS
motorIndexes = top + bot
linkValues   = [0] * len(motorIndexes)
    

visualShapeId                 = -1
nlnk                          = len(link_Masses) # (limiter)
linkVisualShapeIndices        = [-1]        * nlnk    #=[-1,-1,-1, ... , -1]
linkOrientations              = [[0,0,0,1]] * nlnk
linkInertialFramePositions    = [[0,0,0]]   * nlnk
#Note the orientations are given in quaternions (4 params). 
#There are function to convert of Euler angles and back
linkInertialFrameOrientations = [[0,0,0,1]] * nlnk
#Drop the body in the scene at the following body coordinates
# basePosition    = [0,0,0]
baseOrientation = [0,0,0,1]
b_p.Z
robot_positions = [[0,0,b_p.Z]] 
# [[-2,0,b_p.Z],[0,0,b_p.Z],[2,0,b_p.Z]]
for basePosition in robot_positions:
    #Main function that creates the dog
    robot = p.createMultiBody(
        baseMass                      = body_Mass,
        baseCollisionShapeIndex       = base,
        baseVisualShapeIndex          = visualShapeId,
        basePosition                  = basePosition,
        baseOrientation               = baseOrientation,
        
        linkVisualShapeIndices        = linkVisualShapeIndices,
        linkOrientations              = linkOrientations,
        linkInertialFramePositions    = linkInertialFramePositions,
        linkInertialFrameOrientations = linkInertialFrameOrientations,
        
        linkMasses                    = link_Masses,# (limiter)
        linkCollisionShapeIndices     = linkCollisionShapeIndices,# (limiter)
        linkPositions                 = linkPositions, # (limiter, basePosition)
        linkParentIndices             = indices,# (limiter)
        linkJointTypes                = jointTypes,# (limiter)
        linkJointAxis                 = axis # (limiter)
    )
    u.resetCoordinates(robot, motorIndexes, linkValues)

# Add earth like gravity

p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(1)

cameraDistance = 5.0
cameraYaw      = 180
cameraPitch    = -27.60

u.resetCamera(
    cameraDistance = cameraDistance,
    cameraYaw      = cameraYaw,
    cameraPitch    = cameraPitch,
)

TorsoS   = [-15,15];   TorsoF      = [-12,40]; TorsoR      = [-95,95]
Neck     = [-10,40];   HeadR       = [-80,80]; HeadF       = [-60,20]
ShoulderF= [-150, 90]; L_ShoulderS = [0,-150]; R_ShoulderS = [150,0]
ElbwR    = [-110,110]; Elbw        = [120,5]
__HipF   = [-90,35];   R_HipS      = [-90,12]; L_HipS      = [-12,90]; __HipR      = [-90,90]
__Knee   = [ 0,110];   __AnkleF    = [-40,25]; __AnkleS    = [-30,30]

angleFHead        = 0; angleRHead        = 0;       angle_neck   = 0;
angle_TorsoS      = 0; angle_TorsoF      = 0;       angle_TorsoR = 0;

angle_R_ShoulderF = 0; angle_L_ShoulderF = 0
angle_R_ShoulderS = 0; angle_L_ShoulderS = 0
angle_R_ElbowR    = 0; angle_L_ElbowR    = 0
angle_R_Elbow     = 0; angle_L_Elbow     = 0
angle_R_HipF      = 0; angle_L_HipF      = 0
angle_R_HipS      = 0; angle_L_HipS      = 0
angle_R_HipR      = 0; angle_L_HipR      = 0
angle_R_Knee      = 0; angle_L_Knee      = 0
angle_R_AnkleF    = 0; angle_L_AnkleF    = 0
angle_R_AnkleS    = 0; angle_L_AnkleS    = 0


from MAIN_LOOP.printer import printer
from MAIN_LOOP.options import options
from MAIN_LOOP.body_mv import move_head, move_torso, move_arms, move_legs

angleFHead, angleRHead, angle_neck
main_flag = True
while main_flag:
    msg = 'Main: Options(O), BodyMove(M), Info(?), Quit(Q)'
    printer(msg)
    keys = p.getKeyboardEvents()
    if keys.get(111): # o - options
        result = options(cameraDistance, cameraYaw, cameraPitch)
        cameraDistance, cameraYaw, cameraPitch = result
    if keys.get(109): # m - move body paths
        move_flag = True
        while move_flag:
            printer('Move: Head(H), Torso(T), Arms(A), Legs(L), Info(?). Quit(1)')
            move_loop = p.getKeyboardEvents()
            if move_loop.get(104):# H - Head
                angles      = [angle_neck, angleRHead, angleFHead]
                minmaxNandH = [Neck, HeadR, HeadF]
                angles      = move_head(robot, NandH, angles, minmaxNandH)
            if move_loop.get(116):# T - Torso
                angles      = [angle_TorsoS, angle_TorsoF, angle_TorsoR]
                minmaxTorso = [TorsoS,TorsoF,TorsoR]
                angles      = move_torso(robot, Torso, angles, minmaxTorso)
            if move_loop.get(97): # A - Arms
                angles      = [
                    angle_R_ShoulderF, angle_L_ShoulderF,
                    angle_R_ShoulderS, angle_L_ShoulderS,
                    angle_R_ElbowR,    angle_L_ElbowR,
                    angle_R_Elbow,     angle_L_Elbow,]
                minmaxArms  = [ShoulderF, [R_ShoulderS, L_ShoulderS], ElbwR, Elbw]
                Arms        = ShldrF + ShldrS + ElbowR + Elbow
                angles      = move_arms(robot, Arms, angles, minmaxArms)
            if move_loop.get(108):# L - Legs
                angles      = [
                    angle_R_HipF,   angle_L_HipF,
                    angle_R_HipS,   angle_L_HipS,
                    angle_R_HipR,   angle_L_HipR,
                    angle_R_Knee,   angle_L_Knee,
                    angle_R_AnkleF, angle_L_AnkleF,
                    angle_R_AnkleS, angle_L_AnkleS,]
                minmaxLegs = [__HipF, [R_HipS, L_HipS], __HipR, __Knee, __AnkleF, __AnkleS]
                Legs       = HipF + HipS + HipR + Knee + AnkleF + AnkleS
                angles     = move_legs(robot, Legs, angles, minmaxLegs)
            if move_loop.get(49): # Quit MV Block
                move_flag = False
            if move_loop.get(47):
                msg = 'MV Block: press "1" to go "Main"'
                printer(msg)
            time.sleep(0.3)
    if keys.get(47):
        printer(msg)
    if keys.get(113):
        printer('Quit Main...')
        main_flag = False
    time.sleep(0.1)
p.disconnect()

In [ ]:
TorsoS      = [-15,15];   
TorsoF      = [-12,40]; 
TorsoR      = [-95,95]
Neck        = [-10,40];   
HeadR       = [-80,80]; 
HeadF       = [-60,20]
ShoulderF   = [-150, 90]; 
L_ShoulderS = [0,-150]; 
R_ShoulderS = [150,0]
ElbwR       = [-110,110]; 
Elbw        = [120,5]
__HipF      = [-90,35];   
R_HipS      = [-90,12]; 
L_HipS      = [-12,90]; 
__HipR      = [-90,90]
__Knee      = [ 0,110];   
__AnkleF    = [-40,25]; 
__AnkleS    = [-30,30]